In [1]:
import pandas as pd
import geopandas as gpd
import folium
import folium.plugins as plug
import json
from folium.plugins import FloatImage

In [2]:
si=pd.read_excel("적정신호시간횡단보도평가(시).xlsx")
si.head()

,시도명,시군구명,횡단보도관리번호,위도,경도,횡단보도연장,음향신호기설치여부,녹색신호시간,보도턱낮춤여부,점자블록유무,일반인_적정신호시간,교통약자_적정신호시간,평가
0,전라남도,여수시,C0380,34.776244,127.645480,28.0,N,30,Y,Y,35,42,부적당
1,전라남도,여수시,C0381,34.776000,127.645700,28.0,N,30,Y,Y,35,42,부적당
2,전라남도,여수시,C0384,34.778645,127.649215,28.0,N,30,Y,Y,35,42,부적당
3,전라남도,여수시,C0390,34.776325,127.646265,28.0,N,25,Y,Y,35,42,부적당
4,전라남도,여수시,C0391,34.745000,127.674500,30.0,N,30,Y,Y,37,44,부적당


In [3]:
si.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   시도명          1714 non-null   object 
 1   시군구명         1714 non-null   object 
 2   횡단보도관리번호     1714 non-null   object 
 3   위도           1714 non-null   float64
 4   경도           1714 non-null   float64
 5   횡단보도연장       1714 non-null   float64
 6   음향신호기설치여부    1714 non-null   object 
 7   녹색신호시간       1714 non-null   int64  
 8   보도턱낮춤여부      1714 non-null   object 
 9   점자블록유무       1714 non-null   object 
 10  일반인_적정신호시간   1714 non-null   int64  
 11  교통약자_적정신호시간  1714 non-null   int64  
 12  평가           1714 non-null   object 
dtypes: float64(3), int64(3), object(7)
memory usage: 174.2+ KB


In [4]:
gun=pd.read_excel("적정신호시간횡단보도평가(군).xlsx")
gun.head()

,시도명,시군구명,횡단보도관리번호,위도,경도,횡단보도연장,음향신호기설치여부,녹색신호시간,보도턱낮춤여부,점자블록유무,일반인_적정신호시간,교통약자_적정신호시간,평가
0,경상남도,남해군,남해-46,34.771371,127.884684,15,N,20,Y,N,22,26,부적당
1,경상남도,남해군,남해-47,34.755331,127.849633,15,N,20,Y,N,22,26,부적당
2,경상남도,남해군,남해-48,34.761461,127.849163,15,N,20,N,N,22,26,부적당
3,경상남도,남해군,남해-49,34.765817,127.850357,15,N,20,Y,N,22,26,부적당
4,경상남도,남해군,남해-50,34.772702,127.863498,15,N,20,Y,N,22,26,부적당


In [5]:
si_gun=pd.concat([si,gun],axis=0)
#si_gun.to_excel("적정신호시간횡단보도평가(시,군).xlsx") 왜인지는 모르겠으나 concat한 상태의 sigun을 그대로 쓰면 한 값을 불러오면 두개가 리스트로 딸려나와서
#저장한 후 다시 불러와 작업

In [6]:
si_gun=pd.read_excel("적정신호시간횡단보도평가(시,군).xlsx",index_col=0)
si_gun.head()

,시도명,시군구명,횡단보도관리번호,위도,경도,횡단보도연장,음향신호기설치여부,녹색신호시간,보도턱낮춤여부,점자블록유무,일반인_적정신호시간,교통약자_적정신호시간,평가
0,전라남도,여수시,C0380,34.776244,127.645480,28.0,N,30,Y,Y,35,42,부적당
1,전라남도,여수시,C0381,34.776000,127.645700,28.0,N,30,Y,Y,35,42,부적당
2,전라남도,여수시,C0384,34.778645,127.649215,28.0,N,30,Y,Y,35,42,부적당
3,전라남도,여수시,C0390,34.776325,127.646265,28.0,N,25,Y,Y,35,42,부적당
4,전라남도,여수시,C0391,34.745000,127.674500,30.0,N,30,Y,Y,37,44,부적당


---

In [7]:
red=si_gun[si_gun['평가']=="부적당"].reset_index(drop=True)
yellow=si_gun[si_gun['평가']=="교통약자기준 부적당"].reset_index(drop=True)
green=si_gun[si_gun['평가']=="적당"].reset_index(drop=True)

In [8]:
yellow.columns[12]

'평가'

In [9]:
# 시군구 지리정보 불러오기
sigun = "행정경계(시군구)/행정경계(필요한시군구)(5179).geojson"
sigun = gpd.read_file(sigun, encoding='utf-8')
sigun

,SIGUNGU_CD,SIGUNGU_NM,geometry
0,31200,파주시,"MULTIPOLYGON (((927428.936 1995357.109, 927454..."
1,31210,이천시,"MULTIPOLYGON (((997560.746 1928984.155, 997562..."
2,36020,여수시,"MULTIPOLYGON (((983731.378 1557045.622, 983732..."
3,36430,함평군,"MULTIPOLYGON (((918356.658 1693636.061, 918387..."
4,37350,영덕군,"MULTIPOLYGON (((1168961.901 1810372.018, 11689..."
5,38100,양산시,"MULTIPOLYGON (((1141138.272 1727034.957, 11411..."
6,38350,남해군,"MULTIPOLYGON (((1053604.902 1611882.773, 10536..."


In [10]:
center = [36, 128]
map = folium.Map(location=center, zoom_start=7)

In [11]:
folium.GeoJson(sigun, name='sigun').add_to(map)

In [12]:
marker_cluster = plug.MarkerCluster().add_to(map)

In [13]:
for i in range(len(red)):
    iframe ='''<style>th,td {padding:5px;}</style>
    <p style="font-size:15px;text-align:center;">'''+red.columns[2]+" <b>"+red[red.columns[2]][i]+"</b></p>"+'''
    <table border=0 align="center"> 
        <tr>
            <td style=""><b>'''+red.columns[7]+'''</b> </td>
            <td  style="text-align:right;">'''+str(red[red.columns[7]][i])+'''초</td>  
        </tr>
        <tr>
            <td style=""><b>'''+red.columns[10]+'''</b> </td>
            <td  style="text-align:right;">'''+str(red[red.columns[10]][i])+'''초</td> 
        </tr>
        <tr>
            <td style=""><b>'''+red.columns[11]+'''</b> </td>
            <td  style="text-align:right;">'''+str(red[red.columns[11]][i])+'''초</td>
        </tr>
        <tr>
            <td style=""><b>'''+red.columns[12]+'''</b> </td>
            <td  style="text-align:right;color: red;"><b>'''+red[red.columns[12]][i]+'''</b></td>
        </tr>
    </table>'''
    popup = folium.Popup(iframe, min_width=200, max_width=300)
    icon = folium.features.CustomIcon("red-light.png",icon_size=(30, 30))
    folium.Marker([red['위도'][i],red['경도'][i]], popup=popup, icon=icon).add_to(marker_cluster)

In [14]:
for i in range(len(yellow)):
    iframe ='''<style>th,td {padding:5px;}</style>
    <p style="font-size:15px;text-align:center;">'''+yellow.columns[2]+" <b>"+yellow[yellow.columns[2]][i]+"</b></p>"+'''
    <table border=0 align="center"> 
        <tr>
            <td style=""><b>'''+yellow.columns[7]+'''</b> </td>
            <td  style="text-align:right;">'''+str(yellow[yellow.columns[7]][i])+'''초</td>  
        </tr>
        <tr>
            <td style=""><b>'''+yellow.columns[10]+'''</b> </td>
            <td  style="text-align:right;">'''+str(yellow[yellow.columns[10]][i])+'''초</td> 
        </tr>
        <tr>
            <td style=""><b>'''+yellow.columns[11]+'''</b> </td>
            <td  style="text-align:right;">'''+str(yellow[yellow.columns[11]][i])+'''초</td>
        </tr>
        <tr>
            <td style=""><b>'''+yellow.columns[12]+'''</b> </td>
            <td  style="text-align:right;color: #FFC81E;"><b>'''+yellow[yellow.columns[12]][i]+'''</b></td>
        </tr>
    </table>'''
    popup = folium.Popup(iframe, min_width=200, max_width=300)
    icon = folium.features.CustomIcon("yellow-light.png",icon_size=(30, 30))
    folium.Marker([yellow['위도'][i],yellow['경도'][i]], popup=popup, icon=icon).add_to(marker_cluster)

In [15]:
for i in range(len(green)):
    iframe ='''<style>th,td {padding:5px;}</style>
    <p style="font-size:15px;text-align:center;">'''+green.columns[2]+" <b>"+green[green.columns[2]][i]+"</b></p>"+'''
    <table border=0 align="center"> 
        <tr>
            <td style=""><b>'''+green.columns[7]+'''</b> </td>
            <td  style="text-align:right;">'''+str(green[green.columns[7]][i])+'''초</td>  
        </tr>
        <tr>
            <td style=""><b>'''+green.columns[10]+'''</b> </td>
            <td  style="text-align:right;">'''+str(green[green.columns[10]][i])+'''초</td> 
        </tr>
        <tr>
            <td style=""><b>'''+green.columns[11]+'''</b> </td>
            <td  style="text-align:right;">'''+str(green[green.columns[11]][i])+'''초</td>
        </tr>
        <tr>
            <td style=""><b>'''+green.columns[12]+'''</b> </td>
            <td  style="text-align:right;color: green;"><b>'''+green[green.columns[12]][i]+'''</b></td>
        </tr>
    </table>'''
    popup = folium.Popup(iframe, min_width=200, max_width=300)
    icon = folium.features.CustomIcon("green-light.png",icon_size=(30, 30))
    folium.Marker([green['위도'][i],green['경도'][i]], popup=popup, icon=icon).add_to(marker_cluster)

In [16]:
plug.FloatImage('https://github.com/LAB-703/dweer/blob/main/%ED%9A%A1%EB%8B%A8%EB%B3%B4%EB%8F%84%EC%8B%A0%ED%98%B8%EC%8B%9C%EA%B0%84%20%EB%AC%B8%EA%B5%AC.png?raw=true', bottom=3, left=3).add_to(map)

In [17]:
map.save("적정신호시간 클러스터링 시각화.html")